# QDC 2025チャレンジ - トラックA <br> QOBLIB - 市場分割問題

このチャレンジノートでは、量子アルゴリズムを用いて複雑な組合せ最適化問題に取り組む方法を学びます。具体的には、比較的小さなインスタンスでも計算的に難解になる、資源配分の難易度を抱える「マーケット・スプリット問題」を探ります。

このノートは4つのパートに分かれています。
- Part I: マーケットスプリット問題の背景と、[Quantum Optimization Benchmarking Library (QOBLIB) repository](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library) からの問題インスタンスの取得および読み込み方法を学びます。
- Part II: その後、新しい[Qiskit Optimization Mapperアドオン](https://github.com/Qiskit/qiskit-addon-opt-mapper)を使って、問題を量子コンピュータ上で解く二次制約なし二値最適化(QUBO)形式に変換します。
- Part III: [Qiskit Function](https://quantum.cloud.ibm.com/docs/guides/functions)—[Kipu Iskay Optimizer](https://quantum.cloud.ibm.com/docs/guides/kipu-optimization)を用いて問題を解いた例を参照してください。
- Part IV: Challenge に挑む! 市場分割問題の複雑なインスタンスに、Qiskit Functions、カスタムQAOA、あるいはQDC全体で出会う他の高度な手法など、あらゆる量子手法を使って取り組むことになります。

ワクワクする?さあ、始めましょう!

## Table of Contents

* [Imports](#imports)
* [Part I: Understanding and Loading the Market Split Problem (10 points)](#part-I)
* [Part II: QUBO Formulation (10 points)](#part-II)
* [Part III: Solving the problem using Kipu Iskay Optimizer function (0 points)](#part-III)
* [Part IV: Main Challenge — Tackling a Complex Market Split Instance (80 points)](#part-IV)
* [References](#references)
* [Additional Information](#additional-information)
* [Package Versions](#package-versions)

<a id="imports"></a>
## Imports

In [ ]:
# Import common packages
import os
import tempfile
import time
from typing import Tuple, Dict, Optional

import numpy as np
import requests
import matplotlib.pyplot as plt

# Import qiskit classes
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_catalog import QiskitFunctionsCatalog

# Import grading functions
from qc_grader.challenges.qdc_2025.qdc25_lab7 import (
    submit_name,
    grade_kipu_function,
    grade_lab7_ex1,
    grade_lab7_ex2,
    grade_lab7_ex3,
    grade_lab7_ex4,
    grade_lab7_ex5,
    grade_lab7_ex6
)

from qiskit_addon_opt_mapper import OptimizationProblem
from qiskit_addon_opt_mapper.converters import OptimizationProblemToQubo

print("All required libraries imported successfully")
print("Grading functions imported from grader.py")

## Submit Team Name

In [ ]:
# Please use the exact spelling of your team name across teammates and across challenges

team_name = # Add team name string
submit_name(team_name)

<a id="part-I"></a>
## Part I: マーケット分割問題の理解とロード(10点)
### 背景:マーケット分割問題（Market Split Problem）

マーケット分割問題は、一見すると単純でありながら、資源配分において計算論的に非常に困難な課題を提示します。$m$ 種類の商品を扱う企業が、$n$ 個の異なる市場で販売を行っているとします。各市場は特定の商品の組み合わせを購入しており、その情報は行列 $A$ の列として表されます。ビジネス上の目的は、これらの市場を 2 つのバランスの取れた販売地域 に分割し、各地域がすべての商品について総需要のちょうど半分を受け取る ようにすることです。

**数学的定式化:**
求めるのは二値の割当ベクトル $x$ で、次を満たします：
- $x_j = 1$ ：市場 $j$ を地域 A に割り当てる
- $x_j = 0$ ：市場 $j$ を地域 B に割り当てる
- 制約条件として$Ax = b$ を満たす必要があり、ここで $b$ は各商品ごとの目標販売量（通常は総需要の半分）を表します。

**計算量的困難さ:**
ビジネス上の解釈は非常に直感的であるにもかかわらず、この問題は驚くほど計算が困難です。
- **小規模でも解けない**：従来の混合整数計画（MIP）ソルバーでさえ、商品数がわずか 7 のインスタンスに対して、1 時間のタイムアウト以内に解を見つけられないことがあります [3]。
- **指数的な増大**：解の候補は $2^n$ 通りに指数的に増加するため、全探索（ブルートフォース）は現実的ではありません。

このような深刻な計算障壁と、テリトリープランニングや資源配分といった実務的な重要性を併せ持つことから、マーケット分割問題は 量子最適化アルゴリズムの性能評価における理想的なベンチマーク問題となっています。
本ノートブックで用いられている問題インスタンスは、量子および古典的最適化手法を比較するための標準的なテストケースを提供する
[Quantum Optimization Benchmarking Library (QOBLIB) repository](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library) [2]から取得されています。


<a id="data-loading-and-problem-format"></a>
### データの読み込みと問題形式

それでは、問題インスタンスを読み込むための実践的な手順を実装していきましょう。ここでは 2 つの方法があります：

1. **直接ダウンロード**： `fetch_marketsplit_data()` を使って、QOBLIB リポジトリ [2]から直接ダウンロードする方法
2. **手動ダウンロード＋パース**：URL から `.dat` ファイルを手動でダウンロードし、それを `parse_marketsplit_dat()` で解析する方法

どちらの方法でも、問題データを正しく抽出するためには `.dat` ファイルの形式を理解しておく必要があります。

###  .dat ファイル形式の理解

QOBLIB における問題インスタンスは、シンプルなテキスト形式で保存されています。対象とするインスタンス
[`ms_03_200_177.dat`](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/blob/main/01-marketsplit/instances/ms_03_200_177.dat?ref_type=heads)
の実際の内容を見てみましょう。

```python
3 20
  60   92  161   53   97    2   75   81    6  139  132   45  108  112  181   93  152  200  164   51 1002
 176  196   41  143    2   88    0   79   10   71   75  148   82  135   34  187   33  155   58   46  879
  68   68  179  173  127  163   48   49   99   78   44   52  173  131   73  198   64   34  158  102 1040
```

**形式の構造:**
- **1 行目:** `3 20`
  - `3` = 商品数（制約の数／行列 $A$ の行数）
  - `20` = 市場数（変数の数／行列 $A$ の列数）

- **続く 3 行:** 係数行列 $A$ と目標ベクトル $b$
  - 各行には 21 個の数値が含まれています: 最初の 20 個は行列 $A$ の係数, 最後の 1 個は目標値
  - 2 行目: `60 92 161 ... 51 | 1002`
    - 最初の 20 個の数値：20 市場それぞれが「商品 1」をどれだけ販売しているか
    - 最後の数値（1002）：片方の地域に割り当てたい商品 1 の目標販売量
  - 3 行目: `176 196 41 ... 46 | 879`
    - 商品 2 の市場別販売量と目標値（879）
  - 4 行目: `68 68 179 ... 95 | 1040`
    - 商品 3 の市場別販売量と目標値（1040）

**ビジネス的な解釈:**
- 市場 0 は、商品 1 を 60 個、商品 2 を 176 個、商品 3 を 68 個　販売しています。
- 市場 1 は、商品 1 を 92 個、商品 2 を 196 個、商品 3 を 68 個  販売しています。
- このようにして、全 20 市場について販売量が定義されています。
- 目標：これら 20 の市場を 2 つの地域に分割し、商品 1：1002、商品 2：879、商品 3：1040 の販売量を、それぞれの地域が 正確に 得るようにすることです。

### パースの演習

次に、この形式を解析する関数を実装します。重要なポイントは、各行から 係数行列`A` と目標ベクトル`b` を正しく切り出すことです。


<div class="alert alert-block alert-success">
    
<b>Exercise 1: `parse_marketsplit_dat` 関数を完成させなさい（8 点）</b>

あなたの課題は、`.dat` ファイルの各行からデータを正しく抽出するパース処理を完成させることです。

**やるべきこと：**
- `n+1` 個の値を含む行（例：`n=20` のとき `60 92 161 ... 51 1002`）が与えられたら、
- 最初の `n` 個の値を行列 `A` の 1 行として抽出する
- 最後の 1 つの値を、対応するベクトル `b` の要素として抽出する

**ヒント：**
- Python のリストスライスを使いましょう
- 負のインデックスが便利です
- 抽出したデータは、忘れずに `A` と `b` のリストに `append` してください

</div>


In [ ]:
def parse_marketsplit_dat(filename: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Parse a market split problem from a .dat file format.

    See the format structure explained above for details on the file layout.

    Parameters
    ----------
    filename : str
        Path to the .dat file containing the market split problem data.

    Returns
    -------
    A : np.ndarray
        Coefficient matrix of shape (m, n) where m is the number of products
        and n is the number of markets.
    b : np.ndarray
        Target vector of shape (m,) containing the target sales per product.
    """
    with open(filename, "r", encoding="utf-8") as f:
        lines = [
            line.strip()
            for line in f
            if line.strip() and not line.startswith("#")
        ]

    if not lines:
        raise ValueError("Empty or invalid .dat file")

    # First line: m n (number of products and markets)
    try:
        m, n = map(int, lines[0].split())
    except (ValueError, IndexError) as e:
        raise ValueError("Invalid file format: first line must contain 'm n' integers") from e

    if len(lines) < m + 1:
        raise ValueError(f"File contains {len(lines)} lines but expected {m + 1} lines")

    # Next m lines: each row of A followed by corresponding element of b
    A = []
    b = []

    for i in range(1, m + 1):
        try:
            values = list(map(int, lines[i].split()))
        except ValueError as e:
            raise ValueError(f"Invalid integer values in line {i + 1}") from e

        if len(values) != n + 1:
            raise ValueError(
                f"Line {i + 1} contains {len(values)} values but expected {n + 1}"
            )

        ### Don't change any code before this line ###

        ### Write your code below here ###
        # TODO: Append the correct values to A list
        # TODO: Extract the target value and append to b list

        ### Don't change any code past this line ###

    return np.array(A, dtype=np.int32), np.array(b, dtype=np.int32)

In [ ]:
# Submit your answer using following code

grade_lab7_ex1(parse_marketsplit_dat) # Expected result type: Callable

### 直接ダウンロードする関数

この関数は、ダウンロードとパース（解析）の手順を 1 つにまとめたものです。

In [ ]:
def fetch_marketsplit_data(instance_name: str) -> Tuple[Optional[np.ndarray], Optional[np.ndarray]]:
    """
    Fetch market split data directly from the QOBLIB repository.

    This function downloads the .dat file and then uses parse_marketsplit_dat()
    to extract the matrices.

    Parameters
    ----------
    instance_name : str, optional
        Name of the .dat file to fetch from the repository.

    Returns
    -------
    A : np.ndarray or None
        Coefficient matrix of shape (m, n) if successful, None if failed.
    b : np.ndarray or None
        Target vector of shape (m,) if successful, None if failed.
    """
    base_url = (
        "https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/"
        "-/raw/main/01-marketsplit/instances/"
    )
    url = base_url + instance_name

    try:
        # Fetch the file content with timeout
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        # Save to temporary file
        with tempfile.NamedTemporaryFile(
            mode="w", suffix=".dat", delete=False, encoding="utf-8"
        ) as temp_file:
            temp_file.write(response.text)
            temp_file_path = temp_file.name

        try:
            # Use our parsing function to extract A and b
            A, b = parse_marketsplit_dat(temp_file_path)
            return A, b
        finally:
            # Clean up temporary file
            try:
                os.unlink(temp_file_path)
            except OSError:
                pass  # File cleanup failure is not critical

    except requests.RequestException as e:
        print(f"Error fetching data from repository: {e}")
        return None, None
    except (ValueError, IOError, OSError) as e:
        print(f"Error processing data: {e}")
        return None, None

print("Direct download function defined successfully")
print("\nYou can now use either approach:")
print("  1. fetch_marketsplit_data('ms_03_200_177.dat') - downloads and parses automatically")
print("  2. parse_marketsplit_dat('path/to/file.dat') - parses a manually downloaded file")

あるいは、QOBLIB リポジトリから問題インスタンスを直接ダウンロードすることもできます。

**Repository URL**: [https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/tree/main/01-marketsplit/instances?ref_type=heads](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/tree/main/01-marketsplit/instances?ref_type=heads)

<a id="exercise2"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 2: 問題インスタンスの読み込みと分析（2 点）</b>

QOBLIB リポジトリ [2] から、マーケット分割問題のインスタンス
[`ms_03_200_177.dat`](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/blob/main/01-marketsplit/instances/ms_03_200_177.dat?ref_type=heads)
を読み込み、その次元を確認しなさい。

**課題：**

1. `fetch_marketsplit_data()` 関数を使ってリポジトリから直接問題を読み込む、または `.dat` ファイルをダウンロードして`parse_marketsplit_dat()` で解析する
2. 係数行列を変数 `A`、目標ベクトルを変数 `b` に格納する
3. 問題の次元を表示し、商品数が 3、市場数が 20 であることを確認する

</div>

<div class="alert alert-block alert-info">
    
<b>Tips:</b>

- 関数 `fetch_marketsplit_data()` は、`instance_name` という引数を取ります
- 問題インスタンスの `.dat` ファイルをダウンロードした場合は、`parse_marketsplit_dat()` 関数を使って解析してください。この関数はファイルのパスを引数に取るため、正しい相対パスを指定する必要があります
- 行列 `A` の形状は `(m, n)` で、ここで`m` は商品の数、`n` は市場の数を表します
- ベクトル `b` には、各商品ごとの目標販売量（通常は総需要の半分）が格納されます

</div>

In [ ]:
### Write your code below here ###

A, b = 

### Don't change any code past this line ###

In [ ]:
# Submit your answer using following code

grade_lab7_ex2(A, b) # Expected result type: numpy.ndarray, numpy.ndarray

In [ ]:
# Verify problem dimensions and display key information
print("Problem Instance Analysis:")
print("=" * 50)
print(f"Coefficient Matrix A: {A.shape[0]} × {A.shape[1]}")
print(f"   → {A.shape[0]} products (constraints)")
print(f"   → {A.shape[1]} markets (decision variables)")
print(f"Target Vector b: {b.shape[0]} elements")
print(f"   → {b.shape[0]} target sales values (half demand per product)")
print(f"Solution Space: 2^{A.shape[1]} = {2**A.shape[1]:,} possible assignments")

# Display the target sales values
print(f"\nTarget Sales (b): {b}")
print("These represent the desired sales per product for each region.")

<a id="part-II"></a>
## 第 II 部：QUBO 定式化（10 点）

<a id="from-constraints-to-qubo"></a>

### 制約条件から QUBO へ：数学的変換
量子最適化の強みは、制約付き問題を制約なしの二次形式に変換できる点にあります[4]。マーケット分割問題では、次の等式制約

$$ Ax = b $$

（ただし $x ∈ \{0,1\}^n$を、制約違反にペナルティを課すことで QUBO に変換します。

**ペナルティ法：**
ここでは制約 $Ax = b$ を厳密に満たす必要があるため、その違反の二乗を最小化します：

$$f(x) = ||Ax - b||^2$$

この値は、すべての制約が満たされたとき ちょうど 0になります。これを代数的に展開すると、
$$f(x) = (Ax - b)^T(Ax - b) = x^T A^T A x - 2b^T A x + b^T b$$
となります。

**QUBO の目的関数：**
$b^T b$ は定数項なので、最適化問題は次の形に帰着します：
$$\text{minimize} \quad Q(x) = x^T(A^T A)x - 2(A^T b)^T x$$

**重要なポイント:**
この変換は近似ではなく、厳密な変換です。等式制約は自然に二乗され、補助変数や追加のペナルティ係数を導入する必要がありません。そのため、この定式化は数学的に美しく、かつ量子ソルバーにとって計算効率の良い形になります [4]。本ノートブックでは、まず `OptimizationProblem` クラスを使って制約付き問題を定義し、その後 `OptimizationProblemToQubo` を用いて QUBO 形式へ変換します。この変換では、ペナルティ法による処理が自動的に行われます。

<a id="exercise3"></a>
<div class="alert alert-block alert-success">

<b>Exercise 3: QUBO 形式への変換（10 点）</b>

[`qiskit_addon_opt_mapper`](https://github.com/Qiskit/qiskit-addon-opt-mapper) パッケージを用いて、マーケット分割問題をQUBO 形式に変換しなさい。

**課題：**
1. `instance_name`（拡張子 `.dat` を除いたもの）を基にした名前で、`OptimizationProblem` オブジェクトを作成する
2. 問題に二値変数を追加する（行列 `A` の各列／各市場につき 1 つ）
3. 等式制約を追加する（行列 `A` の各行／各商品につき 1 つ、対応する目標値はベクトル `b` から取る）
4. ペナルティ係数を 1 として、問題を QUBO 形式に変換する

**知っておくべきポイント：**
- `OptimizationProblem(name)` を使って問題オブジェクトを作成します
- `.binary_var_list(n)` を使って、`n` 個の二値変数を追加します
- `.linear_constraint(coefficients, sense="==", rhs=value)` を使って、等式制約を追加します
- `OptimizationProblemToQubo(penalty=1).convert(problem)` を使って、QUBO へ変換します

</div>

<div class="alert alert-block alert-info">
    
<b>ヒント:</b>

- 二値変数の数は市場の数（行列 `A` の列数）に等しくなります：`A.shape[1]`
- `A` の各行と、それに対応する `b` の要素についてループを回し、制約を追加します
- 制約 i に対しては、係数として `A[idx, :]` を使用し、右辺の値には `b[idx]` を使います
- 変換後に得られた QUBO は、変数 `qubo` に格納してください

</div>

In [ ]:
### Write your code below here ###

# TODO: Create optimization problem
ms = 

# TODO: Add binary variables (one for each market)


# TODO: Add equality constraints (one for each product)


# TODO: Convert to QUBO with penalty parameter
qubo = 

### Don't change any code past this line ###
print("QUBO Conversion Complete:")
print("=" * 50)
print(f"Number of variables: {qubo.get_num_vars()}")
print(f"Constant term: {qubo.objective.constant}")
print(f"Linear terms: {len(qubo.objective.linear.to_dict())}")
print(f"Quadratic terms: {len(qubo.objective.quadratic.to_dict())}")

In [ ]:
# Submit your answer using following code

grade_lab7_ex3(qubo, A, b) # Expected result type: OptimizationProblem, numpy.ndarray, numpy.ndarray

<a id="part-III"></a>

## Part III: Kipu Iskay Optimizer 関数を用いた問題の解法（0 点）

このパートでは、QUBO 問題を Qiskit Function を使って解く例を紹介します。具体的には、
[Kipu Iskay Optimizer](https://quantum.cloud.ibm.com/docs/guides/kipu-optimization) を利用します。Iskay は、最先端の BF-DCQO algorithm [[1](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.7.L022010),[2](https://doi.org/10.48550/arXiv.2409.04477)] を活用しており、QUBO 形式で表される 制約なし二値最適化問題や、より高次の HUBO問題を解くことができます。BF-DCQO は 非変分型の量子アルゴリズムであり、QAOA のような一般的な変分アルゴリズムに比べて、必要とする計算資源が少ないという特徴があります。

ノートブックの冒頭でも述べたように、最終的なチャレンジへのアプローチは自由です。Qiskit Function を使う方法でも、独自に実装した QAOA でも、あるいは QDC の中で学ぶ他の高度な手法でも構いません。

このセクションは 任意（採点対象外） ですが、ぜひ試してみることを強くおすすめします。Kipu Iskay Optimizer が標準的な QAOA と比べてどのような性能を示すのかを実感でき、その能力について貴重な洞察が得られるはずです。

### Kipu Iskay 量子オプティマイザ関数の読み込み

Kipu Quantum の最適化機能を利用するには、Qiskit Functions Catalog [1] を通じて接続を確立する必要があります。

まず、あなたの認証情報を用いてカタログを初期化します：

In [ ]:
# # If you haven't done so already
# # Uncomment the follow code to save the QDC 2025 account provided to you
#
# your_api_key = "deleteThisAndPasteYourAPIKeyHere"
# your_crn = "deleteThisAndPasteYourCRNHere"

# QiskitRuntimeService.save_account(
#     channel='ibm_quantum_platform',
#     token=your_api_key,
#     instance=your_crn,
#     name="qdc-2025"
# )

In [ ]:
# Load the Qiskit Functions Catalog
catalog = QiskitFunctionsCatalog(name="qdc-2025")

# You should see a list of Qiskit Functions available to you
# If you encounter the error `QiskitServerlessException: Credentials couldn't be verified`,
# it means your access is not yet active
display(catalog.list())

print("Successfully initialized Qiskit Functions Catalog")
print("Ready to load quantum optimization functions")

<a id="function-exercise"></a>
<div class="alert alert-block alert-success">
    
<b>Function Exercise: Iskay 量子オプティマイザ関数の読み込み（0 点）</b>

カタログからKipu Quantum Iskay Quantum Optimizer Function を読み込んでください。


**課題：**
1. Kipu Quantum Iskay Optimizer の正しい関数名を特定する
2. `catalog.load()` を使って、その関数を読み込む
3. 読み込んだ結果を変数 `iskay_solver` に格納する

**知っておくべきポイント：**
- 関数名は `"provider/function-name"` という形式に従います
- Kipu Quantum の Iskay Optimizer の場合、プロバイダは `"kipu-quantum"` です
- 関数名は `"iskay-quantum-optimizer"` です

</div>

<div class="alert alert-block alert-info">
    
<b>ヒント:</b>

- `catalog.load()` メソッドは、文字列としての関数名を引数に取ります
- プロバイダ名と関数名は、スラッシュ `/` で結合します： `"provider/function-name"`
- 読み込まれたオブジェクトには、オプティマイザを実行するための `run()` メソッドが用意されています

</div>

In [ ]:
### Write your code below here ###

function_name = ""  # TODO: Specify the Kipu Quantum Iskay function name
iskay_solver = catalog.load(function_name)

In [ ]:
# Submit your answer for grading
print(grade_kipu_function(iskay_solver))

<a id="converting-qubo-to-iskay-format"></a>
### QUBO を Iskay 形式に変換する

次に、QUBO オブジェクトを Kipu Quantum の Iskay Optimizer が要求する辞書形式 に変換する必要があります。

`problem`および `problem_type` 引数は、次の形式の最適化問題を表現します：

$$
\begin{align}
\min_{(x_1, x_2, \ldots, x_n) \in D} C(x_1, x_2, \ldots, x_n) \nonumber
\end{align}
$$
ここでコスト関数は

$$
C(x_1, ... , x_n) = a + \sum_{i} b_i x_i + \sum_{i, j} c_{i, j} x_i x_j + ... + \sum_{k_1, ..., k_m} g_{k_1, ..., k_m} x_{k_1} ... x_{k_m}
$$

- `problem_type = "binary"`を選ぶと、コスト関数は binary 形式 であることを指定します。つまり$D = \{0,  1\}^{n}$ であり、QUBO/HUBO 形式で書かれていることを意味します。
- 一方、`problem_type = "spin"`を選ぶと、コスト関数は Ising 形式 になります。この場合$D = \{-1, 1\}^{n}$ となります。

問題の係数は、次のように 辞書（dictionary）形式 でエンコードします：
$$
\begin{align} \nonumber
&\texttt{\{} \\ \nonumber
&\texttt{"()"}&: \quad &a, \\ \nonumber
&\texttt{"(i,)"}&: \quad &b_i, \\ \nonumber
&\texttt{"(i, j)"}&: \quad &c_{i, j}, \quad (i \neq j) \\ \nonumber
&\quad  \vdots \\ \nonumber
&\texttt{"(} k_1, ..., k_m  \texttt{)"}&: \quad &g_{k_1, ..., k_m}, \quad (k_1 \neq k_2 \neq \dots \neq k_m) \\ \nonumber
&\texttt{\}}
\end{align}
$$

<div class="alert alert-block alert-warning">
    
<b> 注意事項：</b> 

辞書のキーは、重複しない整数からなる有効なタプルを文字列として表したものでなければなりません。
二値問題では次の関係が成り立ちます：

$$
x_i^2 = x_i
$$

これは $x_i \in \{0,1\}$ なので$x_i \cdot x_i = x_i$となるためです。そのため、QUBO 定式化において線形項 
$b_i x_i$、対角（二次）項 $c_{i,i} x_i^2$の 両方が存在する場合、これらは 1 つの線形係数にまとめる必要があります：

**変数$x_i$の最終的な線形係数は**：$b_i + c_{i,i}$

つまり：
- 線形項`"(i, )"`には、元の線形係数 + 対角二次係数 を入れる
- 対角二次項 `"(i, i)"`は 最終的な辞書に含めては**いけない**
- $i \neq j$の 非対角（二次）項 `"(i, j)"` のみを個別の項として含める

**例:** もし QUBO が $3x_1 + 2x_1^2 + 4x_1 x_2$ であれば、Iskay 用の辞書は：
- `"(0, )"`: `5.0` ($3 + 2 = 5$を合算)
- `"(0, 1)"`: `4.0` (非対角項)

`"(0, )"`: `3.0` と `"(0, 0)"`: `2.0`のように分けては**いけません**。
    
</div>

<a id="exercise4"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 4: QUBO を Iskay 形式に変換（0 点）</b>

QUBO オブジェクトを、Kipu Quantum の Iskay Optimizer が要求する辞書形式 に変換しなさい。

**課題:**
1. キー `"()"`を使って 定数項 を持つ辞書 `iskay_input` を作成する
2. キー `"(i, )"`（閉じ括弧の前にスペースがある点に注意）形式で 線形係数 を追加する
3. キー `"(i, j)"` 形式で 二次係数 を追加する

**知っておくべきポイント:**
- 定数項： `qubo.objective.constant`
- 線形項： `qubo.objective.linear` (enumerate を使うとインデックスと値を取得できます)
- 二次項： `qubo.objective.quadratic.to_dict()` (タプルキーの辞書が返されます)
- すべての値は float 型に変換 してください
- 二次項のキーは `str(key)` を使って タプル → 文字列 に変換します

</div>

<div class="alert alert-block alert-info">
    
<b>ヒント:</b>

- まず空の辞書を作成し、定数項を追加しましょう： `iskay_input = {"()": qubo.objective.constant}`
- 複数のエントリを一度に追加するには、辞書の `.update()`メソッドが便利です
- 線形項には、辞書内包表記を使います： `{f"({idx}, )": float(val) for idx, val in enumerate(qubo.objective.linear)}`
- 二次項には、次のように書けます： `{str(key): float(val) for key, val in qubo.objective.quadratic.to_dict().items()}`
- f文字列 `f"({idx}, )"` を使うと、閉じ括弧の前にスペースを含んだ正しい形式 のキーを作れます

</div>

In [ ]:
### Write your code below here ###

iskay_input = {}

# TODO: Add constant term
iskay_input = {"()": qubo.objective.constant}

for i in range(qubo.get_num_vars()):
    for j in range(i, qubo.get_num_vars()):
        if i == j:
            # TODO: Add linear term (including diagonal quadratic contribution)
            iskay_input[f"({i}, )"] = 
        else:
            # TODO: Add off-diagonal quadratic term
            iskay_input[f"({i}, {j})"] = float(qubo.objective.quadratic.to_dict().get((i, j)))


In [ ]:
# Submit your answer using following code

grade_lab7_ex4(problem=iskay_input, qubo=qubo)# Expected result type: dict, OptimizationProblem

In [ ]:
print("Iskay Dictionary Format:")
print("=" * 50)
print(f"Total coefficients: {len(iskay_input)}")
print(f"  • Constant term: {iskay_input['()']}")
print(f"  • Linear terms: {sum(1 for k in iskay_input.keys() if k != '()' and ', )' in k)}")
print(f"  • Quadratic terms: {sum(1 for k in iskay_input.keys() if k != '()' and ', )' not in k)}")
print("\nSample coefficients:")

# Get first 10 and last 5 items properly
items = list(iskay_input.items())
first_10 = list(enumerate(items[:10]))
last_5 = list(enumerate(items[-5:], start=len(items)-5))

for i, (key, value) in first_10 + last_5:
    coeff_type = "constant" if key == "()" else "linear" if ", )" in key else "quadratic"
    print(f"  {key}: {value} ({coeff_type})")
print("  ...")
print("\n✓ Problem ready for Iskay optimizer!")

### 変換の理解

`qiskit_addon_opt_mapper`パッケージを用いて、マーケット分割問題を QUBO 形式へ正常に変換 できました。
この変換によって、次のものが作成されています：
- 二値変数と線形制約を持つ `OptimizationProblem` 
- 二次項・線形項・定数項を含む QUBO 表現
- 量子最適化に利用できる Iskay 互換の辞書形式

Kipu Quantum's Iskay Optimizer.これで、`iskay_input` 辞書には、Kipu Quantum の Iskay Optimizer 上で最適化を実行するために必要なすべての情報 が含まれています。

<a id="understanding-iskay-input-parameters"></a>
### Iskay 入力パラメータの理解

Iskay Optimizer が最適化問題を効果的に解くためには、いくつかの重要なパラメータを指定する必要があります。ここでは、それぞれのパラメータと量子最適化プロセスにおける役割を見ていきます:

#### 必須パラメータ

| パラメータ | 型 | 説明 | 例 |
|-----------|------|-------------|---------|
| **problem** | `Dict[str, float]` | 文字列キー形式で表された QUBO の係数 | `{"()": -21.0, "(0,4)": 0.5, "(0,1)": 0.5}` |
| **problem_type** | `str` | Ising形式の指定：QUBO の場合は`"binary"`、Ising の場合は `"spin"`| `"binary"` |
| **backend_name** | `str` | 使用する量子デバイス | `"ibm_boston"` |

#### 重要なポイント

- **問題形式**: 今回は変数が二値（0/1）で市場の割り当てを表しているため、`"binary"`  を使用します。
- **バックエンドの選択**: 利用可能な QPU（例：`"ibm_boston"`）の中から、目的や計算リソースの割り当てに応じて選択します。
- **QUBO 構造**: 問題辞書には、数学的変換によって得られた 正確な係数 がそのまま含まれています。

### 高度なオプション（任意）

Iskay には、オプションパラメータによる 細かなチューニング機能 が用意されています。
ほとんどの問題ではデフォルト設定で十分うまく動作しますが、特定の要件に合わせて挙動をカスタマイズすることも可能です:
| パラメータ                    | 型           | デフォルト | 説明                             |
| ------------------------ | ----------- | ----- | ------------------------------ |
| **shots**                | `int`       | 10000 | 1 イテレーションあたりの量子測定回数（多いほど精度向上）  |
| **num_iterations**       | `int`       | 10    | アルゴリズムの反復回数（多いほど解の質が改善する可能性あり） |
| **use_session**          | `bool`      | True  | IBM セッションを使用して待ち時間を短縮          |
| **seed_transpiler**      | `int`       | None  | 回路コンパイル結果を再現可能にするための乱数シード      |
| **direct_qubit_mapping** | `bool`      | False | 仮想量子ビットを物理量子ビットへ直接マッピング        |
| **job_tags**             | `List[str]` | None  | ジョブ管理用のカスタムタグ                  |
| **preprocessing_level**  | `int`       | 0     | 問題の前処理強度（0–3）※下記参照             |
| **postprocessing_level** | `int`       | 2     | 解の後処理（洗練）レベル（0–2）※下記参照         |
| **transpilation_level**  | `int`       | 0     | トランスパイラ最適化レベル（0–5）※下記参照        |
| **transpile_only**       | `bool`      | False | 実行せずに回路最適化のみ解析                 |


**Preprocessing Levels (0-3)**: 特に 大規模問題（ハードウェアのコヒーレンス時間に収まらない問題）で重要です。
レベルを上げるほど 近似を導入して回路深さを浅く します:
- **Level 0**: 厳密（ただし回路は長くなる）
- **Level 1**: 精度と近似のバランス良好（角度が下位 10% のゲートを削除）
- **Level 2**: やや強い近似（下位 20% を削除、`approximation_degree=0.95`）
- **Level 3**: 最大近似（下位 30% を削除、`approximation_degree=0.90`）

**Transpilation Levels (0-5)**: 量子回路コンパイル時の 高度な最適化の試行回数 を制御します。
その分、古典計算コストは増える可能性があります。また、場合によっては回路深さがあまり変わらないこともあります。
デフォルトの Level 2 は、一般に 最小回路＋比較的高速 でバランスが良い設定です。
- **Level 0**: 分解済み DCQO 回路の最適化（配置・ルーティング・スケジューリング）
- **Level 1**: PauliEvolutionGate → DCQO 回路（max_trials=10）
- **Level 2**: PauliEvolutionGate → DCQO 回路（max_trials=15）
- **Level 3**: PauliEvolutionGate → DCQO 回路（max_trials=20）
- **Level 4**: PauliEvolutionGate → DCQO 回路（max_trials=25）
- **Level 5**: PauliEvolutionGate → DCQO 回路（max_trials=50）

**Postprocessing Levels (0-2)**: 古典的な後処理による 解の改良（ビット反転誤差の補正など） の強さを制御します。
ローカルサーチの greedy パス回数を増やします:
- **Level 0**: 1 pass
- **Level 1**: 2 passes
- **Level 2**: 3 passes

**Transpile-Only モード**: 量子アルゴリズムを実行せず、回路最適化のみを解析したい場合 に利用できます。

### カスタム設定の例

以下は、Iskay をさまざまな設定で構成する方法の一例です:

In [ ]:
# Example: Custom configuration
custom_options = {
    "shots": 15_000,                                        # Higher shot count for better statistics
    "num_iterations": 12,                                   # More iterations for solution refinement
    "preprocessing_level": 1,                               # Light preprocessing for problem simplification
    "postprocessing_level": 2,                              # Maximum postprocessing for solution quality
    "transpilation_level": 3,                               # Using higher transpilation level for circuit optimization
    "seed_transpiler": 42,                                  # Fixed seed for reproducible results
    "job_tags": ["market_split"]                            # Custom tracking tags
}

print("Custom Configuration:")
for key, value in custom_options.items():
    print(f"  {key}: {value}")

### デフォルト設定

ここでは、学習目的と実行時間のバランスを取った、シンプルな設定を使用します。

In [ ]:
# We've already created the iskay_input dictionary in the previous section
# Now we just need to verify it's ready for use

print("Iskay Optimizer Configuration Ready:")
print("=" * 40)
print(f"  problem: QUBO with {len(iskay_input)} coefficients")
print(f"  problem_type: binary (0/1 variables)")
print(f"  backend_name: To be specified in Exercise 3")
print(f"  options: To be configured in Exercise 3")

<a id="exercise5"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 5: Iskay Optimizer を設定して実行する (0 点)</b>

あなたの認証情報を用いて Iskay Optimizer を設定し、最適化ジョブを送信しましょう。

**タスク:**
1. `iskay_input` 辞書を以下の内容で完成させる：
   - インスタンス `'ms_03_200_177.dat'` に対応するあなたの QUBO 問題
   - 問題タイプは `binary`
   - バックエンド名は `"ibm_pittsburgh"`
   - カスタムオプションとして：
      - `num_iterations=5`
      - `transpilation_level=2`

2. `iskay_solver.run()`を使ってジョブを送信する
3. ジョブの完了を待ち、結果を取得する

</div>

<div class="alert alert-block alert-warning">
<b>セッション最大時間</b>

デフォルトのセッション最大時間は 990秒（16.5分） に設定されています。この制限時間を超えると、それ以降に送信されるジョブは失敗します。このデフォルトのセッション制限は上書きすることも可能ですが、QDC チャレンジでは すべての参加者が公平に QPU を利用できるようにするため、デフォルトのタイムアウトを使用してください。この時間制限内でも、十分に競争力のあるスコアを達成できるはずです。

</div>

In [ ]:
### Write your code below here ###

problem = iskay_input  # Use the iskay_input dictionary we created earlier
problem_type = 
backend_name = 
options = 

### Don't change any code past this line ###

iskay_run_input = {
    "problem": problem,
    "problem_type": problem_type,
    "backend_name": backend_name,
    "options": options
}

In [ ]:
# Submit your answer using following code (check configuration before running on QPU)

grade_lab7_ex5(iskay_run_input) # Expected result type: dict

### 量子最適化の実行

設定が graderのチェックをpassしたら、Kipu Quantum のインフラストラクチャに最適化問題を送信できます。この最適化プロセスでは、量子アルゴリズムを用いて解空間を効率的に探索し、高品質な市場割り当て（market assignments）を見つけます。

In [ ]:
# Submit the optimization job to Kipu Quantum's Iskay optimizer
print("Submitting optimization job to Kipu Quantum...")
print(f"Problem size: {A.shape[1]} variables, {len(iskay_input)} QUBO terms")

job = iskay_solver.run(**iskay_run_input)
job_id = job.job_id
print(f"Submitted to IBM with job ID: {job_id}")

Wait until the job is finished:

In [ ]:
while True:
    print(f"Waiting for job {job_id} to complete... (status: {job.status()})",  end='\r', flush=True)
    if job.status() in ['DONE', 'CANCELED', 'ERROR']:
        print(f"Job {job_id} completed with status: {job.status()}")
        break
    time.sleep(30)

print(f"\nJob {job_id} finished!")

# Retrieve the optimization results
result = job.result()
print("Optimization complete!")

### 量子最適化の出力を理解する

Iskay Optimizer は、量子レベルでの洞察（quantum-level insights） と ビジネスで実際に活用できる解（business-actionable solutions） の両方を提供する、包括的な結果を返します。

#### Result ディクショナリの構造

| フィールド           | 型      | 説明                             |
| --------------- | ------ | ------------------------------ |
| `solution`      | `Dict` | 最適解における変数の割り当て                 |
| `solution_info` | `Dict` | ビット列・コスト・マッピングなどの詳細情報          |
| `prob_type`     | `str`  | 問題の定式化タイプ（`binary` または `spin`） |

#### Solution info ディクショナリ

| フィールド             | 型                | 説明                    |
| ----------------- | ---------------- | --------------------- |
| `bitstring`       | `str`            | 量子デバイスから得られた最適ビット列    |
| `cost`            | `float`          | 目的関数の値                |
| `mapping`         | `Dict[int, int]` | ビット列と変数番号の対応関係        |
| `seed_transpiler` | `int`            | 再現性確保のためのトランスパイラのシード値 |

In [ ]:
# Display the optimization results
print("Optimization Results")
print("=" * 50)
print(f"Problem Type: {result['prob_type']}")
print(f"\nSolution Info:")
print(f"  Bitstring: {result['solution_info']['bitstring']}")
print(f"  Cost: {result['solution_info']['cost']}")
print(f"\nSolution (first 10 variables):")
for i, (var, val) in enumerate(list(result['solution'].items())[:10]):
    print(f"  {var}: {val}")
print("  ...")

<a id="part-IV"></a>
## Part IV: Main Challenge — 複雑な Market Split インスタンスへの挑戦（80点）

このノートブックのメインチャレンジへようこそ — あなたの 量子問題解決スキルの究極テスト です！

<a id="exercise6"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 6: 複雑な Market Split インスタンスに挑戦（80点）</b>
 
あなたのミッションは、QOBLIB に含まれる Market Split 問題の難しいインスタンス
[`ms_05_100_003`](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/blob/main/01-marketsplit/instances/ms_05_100_003.dat?ref_type=heads) に対して、実行可能（feasible）な解を見つけること です。

標準的な QUBO 定式化と QAOA アルゴリズムを用いると、このインスタンスは、40量子ビットが必要で、数千個の2量子ビットゲートが必要となり、現在の量子ハードウェアの限界に迫る規模 になります。
ここでこそ、あなたの創造力と最適化スキルが真価を発揮します！

以下を含め、どんな量子手法でも自由に試して構いません： 
- Qiskit Functions
- カスタム QAOA 実装
- 高度なトランスパイルや回路最適化戦略
- 別の問題定式化
- 古典的な前処理・後処理テクニック

アイデア集として [Quantum Optimization Best Practices repository](https://github.com/qiskit-community/qopt-best-practices) も参考にしてください。

このインスタンスの参照解（reference solution）は QOBLIB に用意されていますが、あなたの目標は 既知の答えを使うことではありません。目指すべきは、一般的で、スケーラブルで、同程度の複雑さ（例：他の `m=5` インスタンス）にも対応できる汎用的な解法を設計すること です。

> ⚠️ **重要:** 既知の正解を直接利用したり依存したりする解法は 失格 となります。求められているのは、本物の量子的イノベーション、創造的・アルゴリズム的アプローチ、量子計算の限界に挑戦する解法です。

提出期限後、上位スコアのノートブックは専門家パネルによってレビューされ、追加の問題インスタンスを実行して結果が検証されます。詳細は[QDC contest rules](https://github.com/qiskit-community/qdc-challenges-2025/contest_rules.md)を参照してください。

さあ、あなたの番です。常識にとらわれず、限界に挑み、あなたの量子ツールキットの実力を見せてください！

</div>

<div class="alert alert-block alert-warning">
    
<b>スコア計算式</b>

スコア関数は次のように定義されます：

$$
\text{score} = 80 \times \left( 1 - \min\left(1, \frac{\text{violation}}{200}\right) \right)^4
$$

ここで、`x` は候補解（candidate solution）、`A` と `b` は問題インスタンスを定義する係数行列とターゲットベクトルです。

**violation** は、解がどれだけ制約を満たしていないか（制約違反の度合い）を表す指標で、次のように計算されます：

$$
\text{violation} = \sum_{i} \left( (A x - b)_i \right)^2
$$ 

violation が 小さいほど、制約をよく満たしている（より feasible）。そしてスコアは、4乗ペナルティ（^4）がかかっているため、少しの違反でも急激にスコアが下がる設計で、ほぼ実行可能な解を強く優遇する設計になっています。

violation が **200** を超えた場合、スコアは **0** に設定されます。一方、達成可能な最大スコアは **80** です。
</div>

In [ ]:
### Write your code below here ###
# Following the Qiskit pattern as a suggestion

# Map the problem instance to quantum circuits and operators



# Optimize for target hardware



# Execute on target hardware



# Post-process results


# solution_bitstring is a binary string of length 40
solution_bitstring = 

### Don't change any code past this line ###

In [ ]:
# Submit your answer using following code

grade_lab7_ex6(solution_bitstring) # Expected result type: str

<a id="references"></a>
## References

[1] **Kipu Quantum Iskay Optimizer**  
IBM Quantum Functions Catalog. [https://docs.quantum.ibm.com/guides/functions](https://docs.quantum.ibm.com/guides/functions)

[2] **QOBLIB - Quantum Optimization Benchmarking Library**  
Zuse Institute Berlin (ZIB). [https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library)  
Repository of standardized benchmark instances for quantum and classical optimization algorithms.

[3] **Lodi, A., Tramontani, A., & Weninger, K. (2023)**  
"The Intractable Decathlon: Benchmarking Hard Combinatorial Problems."  
*INFORMS Journal on Computing*.  
Features the Market Split problem as one of ten challenging combinatorial optimization benchmarks.

[4] **Glover, F., Kochenberger, G., & Du, Y. (2019)**  
"Quantum Bridge Analytics I: A Tutorial on Formulating and Using QUBO Models."  
*4OR: A Quarterly Journal of Operations Research*, 17(4), 335-371.  
Comprehensive tutorial on QUBO formulations for optimization problems.

<a id="additional-information"></a>
## Additional Information

**Created by:** Murilo Henrique de Oliveira

**Advised by:** Junye Huang, Daniel Egger

**Version:** 1.0.0

<a id="package-versions"></a>
## Package Versions

In [ ]:
import qiskit
import qiskit_ibm_runtime
import qiskit_ibm_catalog

print(f'Qiskit: {qiskit.__version__}')
print(f'Qiskit IBM Runtime: {qiskit_ibm_runtime.__version__}')
print(f'Qiskit IBM Catalog: {qiskit_ibm_catalog.__version__}')
print(f'NumPy: {np.__version__}')